## Create dataset

Do all imports.

In [ ]:
# For numerical methods
import numpy as np

# For image processing and visualization of results
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch

Function to print things nicely.

In [ ]:
def myprint(M):
    if M.shape:
        with np.printoptions(linewidth=150, formatter={'float': lambda x: f'{x:10.4f}'}):
            print(M)
    else:
        print(f'{M:10.4f}')

Load two images from video.

In [ ]:
# Specify filename
video_filename = 'video.MOV'

# Create a video reader
video_src = cv2.VideoCapture(video_filename)

# Say what frames we want to read
# - index of first frame
i_frame_1 = 0
# - index of second frame
i_frame_2 = 30

# Read first frame
video_src.set(cv2.CAP_PROP_POS_FRAMES, i_frame_1)
success, frame = video_src.read()
assert(success)
img1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Read second frame
video_src.set(cv2.CAP_PROP_POS_FRAMES, i_frame_2)
success, frame = video_src.read()
assert(success)
img2 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

Detection.

In [ ]:
# Create a SIFT feature detector
sift = cv2.SIFT_create()

# Apply detector to find keypoints (pts) and descriptors (desc) in each image
pts1, desc1 = sift.detectAndCompute(image=img1, mask=None)
pts2, desc2 = sift.detectAndCompute(image=img2, mask=None)

Matching.

In [ ]:
# Create a brute-force matcher
bf = cv2.BFMatcher(
    normType=cv2.NORM_L2,
    crossCheck=False,       # <-- IMPORTANT - must be False for kNN matching
)

# Function to get good matches with ratio test
def get_good_matches(descA, descB, threshold=0.5):
    # Find the two best matches between descriptors
    matches = bf.knnMatch(descA, descB, k=2)

    # Find the subset of good matches
    good_matches = []
    for m, n in matches:
        if m.distance / n.distance < threshold:
            good_matches.append(m)
    
    # Return good matches, sorted by distance (smallest first)
    return sorted(good_matches, key = lambda m: m.distance)

# Match the two images
matches = get_good_matches(desc1, desc2)
print(f'found {len(matches)} good matches')

Visualize all good matches.

In [ ]:
# Create figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 10))

# Show images
ax1.imshow(img1, cmap='gray')
ax2.imshow(img2, cmap='gray')

# Show matches
for m in matches:
    # - Get location of keypoints
    p1 = pts1[m.queryIdx].pt
    p2 = pts2[m.trainIdx].pt
    # - Draw line connecting keypoint in first image with keypoint in second image
    fig.add_artist(
        ConnectionPatch(
            p1, p2, 
            'data', 'data',
            axesA=ax1, axesB=ax2,\
            color='red',
            connectionstyle='arc3, rad=0.',
            linewidth=0.1,
        )
    )
    # - Draw red dot at each keypoint
    ax1.plot(p1[0], p1[1], 'r.', markersize=2)
    ax2.plot(p2[0], p2[1], 'r.', markersize=2)

plt.show()

Define camera matrix.

In [ ]:
K = np.array([[1565.7702703272157, 0.0, 964.2389356041999], [0.0, 1562.3561924508267, 537.4247202074102], [0.0, 0.0, 1.0]])

Get image coordinates of matches.

In [ ]:
# Create a, b
a = []
b = []
for m in matches:
    a.append(pts1[m.queryIdx].pt)
    b.append(pts2[m.trainIdx].pt)
a = np.array(a)
b = np.array(b)

## Try two-view reconstruction

Do the following things:

* Apply your own code for two-view reconstruction and confirm failure.
* Apply code from OpenCV for two-view reconstruction and confirm success.
* List any reason you can think of for why your code fails and OpenCV code succeeds.
* Change your code so that it at least runs without error and produces a result (how different is that result from OpenCV?).
* Change your code so that it produces results that are more consistent with those from OpenCV.